In [ ]:
import numpy as np
import torch
from typing import List

# Symmetries in RL - Practical


Aims of the practical
- understand symmetries in MPD
- design policies that encode these symmetries (with less parameters)
- try them out (if time allows)

## Introduction 

Policy gradients (PG) allow for a great flexibility in the parameterization of policies to solve MDPs.

Policy gradients algorithms rely on the PG theorem which expresses the gradient of the RL loss (here expected sum of discounted future loss $V_{\theta} = \mathbb{E}[\sum_i^{\infty} \gamma^i r_i]$) in a form amenable to sample estimation.
Noting $\theta\in \Theta$ the paramaters of a class of policies $\{\pi_{\theta} \mid \theta \in \Theta \}$.

$$\nabla_\theta \, V_{\theta} = \mathbb{E}_{\pi} [Q^\pi(s,a) \nabla_{\theta} \log\,\pi(a|s)]$$

In PG algorithms, iterative updates to $\theta$ are made by following (an estimate of) the negative gradient $g_{\theta} = -\nabla_\theta \, V_{\theta}$ until convergence.

$$\theta \leftarrow \theta + \alpha g_{\theta}$$


In particular, it can be used to encode prior information about the system into the policy.

The aim of this notebook is to explore ways to introduce *symmetry* assumptions about the MDP into the policy.
This notebook is based on the paper: [MDP Homomorphic Networks: Group Symmetries in Reinforcement Learning](https://arxiv.org/abs/2006.16908) by van der Pol et al.

## Symmetries in RL: a brief intro

Cartpole example
* state $s = [x, \dot{x}, \theta, \dot{\theta}]$ (cart position, pole angle, derivatives)
* action $a \in \{\leftarrow, \rightarrow\}$, (lateral force on the cart)

There is a symmetry around $s_c = [0,0,0,0]$.
Consider 
* the reflexion operator $L[s] = -s$
* the swap operator on a binary policy $\pi=[p, 1-p]$:  $K[\pi] = [1-p, p]$

The optimal policy $\pi^*$ can be shown to satisfy
$$ K[\pi^*(s)] = \pi^*(L[s])$$

In other words
$$\pi^*(\leftarrow|s) = \pi^*(\rightarrow|-s) $$

<img src="pictures/mdp_hom.png" width=300  />


The consequence is you interactions on both sides of the point of symmetry can inform the same simpler policy $\bar{\pi}$ defined for  $s \in (\mathbb{R}^{+})^{4}$, and lead to better **sample efficiency**.

## Understanding Invariance and Equivariance

### Prelimimary : Group

you may want a refresher: [wikipedia](https://en.wikipedia.org/wiki/Group_(mathematics)#Definition)

---

Let $G$ be a group indexing a set of transformations operators $L_g : X \to X$, $g \in G$ 

Let $f$ be a mapping from $X$ to $Y$

### Invariance

$f$ is invariant or symmetric to $L_g$ if $f(x) = f(L_g[x])$, for all $g \in G$, $x \in X$

$\{L_g\}_{g\in G}$ is a set of symmetries of $f$ 

For example, convolutional networks are invariant to translation of the input.



### Equivariance

$f$ is equivariant to $L_g$ if there exists a __second__ transformation operator $K_g : Y \to Y$ in the output space of $f$ such that 

$\quad K_g[f(x)] = f (L_g [x])$, for all $g \in G, x \in X$ .


This is a good property to have in image segmentation models with respect to translations and rotations (with $K_g=L_g$).


## Identifying the Symmetries of an MDP

**MDP with symmetries**. 

In an MDP with symmetries there is a set of transformations on the state-action space, which leaves the reward function and transition operator invariant. We define a state transformation and a state-dependent action transformation as $L_g : S \to S$ and $K_g^s : A \to A$ respectively. Invariance of the reward
function and transition function is then characterized as

$\quad\quad R(s, a) = R(L_g [s], K^s_g [a])$ for all $g \in G, s \in S, \in  A$ 

$\quad\quad T (s′|s, a) = T (L_g [s′]|L_g [s], K^s_g [a])$ for all $g \in G, s \in S, a \in A.$ 


For the cartpole example, there are 2 pairs of input / output transformations to that leave the optimal policy unchanged

<img src="pictures/mdp_hom_half.png" width=500  />


**Question**: can you figure and formalize these?

- the identity / identity pair.
- the reflexion around $s_c= (0,0,0,0)$ / swap of policy outcomes

Let's code these up. Both of these can be written, respectively, as matrix operations on the state and policy output vector.


In [ ]:
def get_cartpole_state_group_representations() -> List[torch.TensorType]:
    """
    Matrix representation of the group symmetry on the state: 
    * identity
    * a multiplication of all state variables by -1
    
    return: a list of two 4*4 matrices
    """
    raise NotImplementedError
    
def get_cartpole_action_group_representations() -> List[torch.TensorType]:
    """
    Representation of the group symmetry on the policy: 
    * identity
    * a permutation of the actions
    
    return: a list of two 2*2 matrices
    """
    raise NotImplementedError


Let's test state and action group representations

In [ ]:
# This cell just checks the function you coded work as intended.
# More precisely it checks that transformations form a group and can be composed.

state_group_reps = get_cartpole_state_group_representations()
action_group_reps = get_cartpole_action_group_representations()

# picks two indices randomly
i,j = np.random.randint(0, len(state_group_reps)-1, size=2)

# check that composition of 2 elements in the group, stays in the group
for group_rep in [state_group_reps, action_group_reps]:
    element1 = group_rep[i]
    element2 = group_rep[j]
    
    composed_element = your code here (use torch.matmul)
    
    assert any([torch.equal(composed_element, g) for g in group_rep])


## Building Equivariant Layers for a policy


### Goal

Having identified the pairs of transformations, we know the optimal policy will satisfy the equivariance property: 

$$ K_g[\pi^*(s)] = \pi^*(L_g[s]), \forall g \in G$$

we can build a neural network layer satisfying this property with fewer parameters and better generalization properties than a network that does not make this assumption

### First step: Linear Network

Classic (linear) Neural network layer: $ z' = W z + b$. 

To simplify the math, we merge the bias into the weights so $W \to [W, b]$ and $z \to [z, 1]$. This leads to:

$$z' = W z$$

For a given pair of linear group transformation operators in matrix form $(L_g , K_g)$, where $L_g$ is the input transformation and $K_g$ is the output transformation, we then have to solve the equation

$$K_g W z = W L_g z, \forall g \in G, z$$


Space of Equivariant weights
$$W_{eq} = \{ W ∈ W_{total} | K_g W = W L_g , \forall g \in G\}$$


Symmetrizer of weights
$$S(W) = \frac{1}{|G|}\sum_{g\in G} K_g^{−1}  W L_g $$

We have that
$$\forall W \in W_{total}, S(W) \in W_{eq}$$


**What about the non-linearity?**: 
Point-wise nonlinearity do not change the equivariance property for permuations!

**Questions:**
* if we use $[\phi_1(s), \phi_2(s)] = W s + b$ (a linear network), what is the size of $W$? 
* What is the dimension of $W_{total}$?
* What about the dimension of $W_{eq}$?


In [ ]:
# Turn a pre-existing neural network layer into an equivariant layer 
# via 'symmetrization' given the identified invariances

def symmetrize(W: torch.TensorType, group: List[torch.TensorType]) -> torch.TensorType:
    """
    Create equivariant weight matrix
    INPUT
    :param W: input weight of size 2 x 4
    :group: the invariance representation
    OUTPUT
    the symmetrized weight of size 2 x 4
    """
    
    raise NotImplementedError
        
        
# Let's check shapes
group = [get_cartpole_state_group_representations(),
         get_cartpole_action_group_representations()]
W = torch.tensor(np.random.rand(2, 4).astype(np.float32))
W_sym = symmetrize(W, group)

assert W.shape == W_sym.shape

print('original weights:')
print(W)
print('symmetrized weights:')
print(W_sym)


Let's check the symmetrized weights do indeed parameterize a policy with the desired equivariance property

In [ ]:
# use the following helper function
def test_network_is_equivariant(W: torch.TensorType, z: torch.TensorType, group: List[torch.TensorType]) -> None:
    """ testing for a specific input z 
    INPUT
    :param W: input weight of size 2 x 4
    :param z: input state vector of size 4
    :param group: the invariance representation
    """
    
    is_equivariant = []
    for i in range(len(group[0])):

        # here we had bias and non-linearity
        b = 1 # bias
        phi = lambda x: torch.relu(x) # pointwise non linearity
        
        L_g = group[0][i] # input transformation
        Lz = torch.matmul(L_g, z)
        WLz = phi(torch.matmul(W, Lz) + b)
        Wz = phi(torch.matmul(W, z) + b)
        K_g = group[1][i] # output transformation
        KWz = torch.matmul(K_g, Wz)

        is_equivariant.append(torch.equal(KWz, WLz))
    assert all(is_equivariant)
    print('network is equivariant!')

    
# sample random state
z = torch.tensor(np.random.rand(4,1).astype(np.float32))

# use the test function on this state for the symmetrized weights
test_network_is_equivariant(W_sym, z, group) # test should pass!


## Building a basis for Equivariant layers

When working with discrete state actions, the set of equivariant weight
$$W_{eq} = \{ W ∈ W_{total} | K_g W = W L_g , \forall g \in G\}$$

is a linear subspace of the space of weights $W_{total}$.

To parameterize an equivariant layer, it is enough to express the weights in a basis of $W_{eq}$


**How to find a basis for $W_{eq}$?**
One procedure consists in sampling enough weights from $W_{eq}$ such that their span (almost surely) equals $w_{eq}$, in other words, they cover all directions of $W_{eq}$. If such a sampling procedure is available, then the problem boils down to finding a basis for a collection of vectors, which is a simpler problem.


A basis for $W_{eq}$ can be constructed using the following procedure
* 1) sample non-equivariant weights  $(W_n)_{n=1..N}$
* 2) symmetrize those weights $\tilde{W}_n = S(W_n)$
* 3) find a basis for $W_{eq}$ from $(\tilde{W}_n)_{n=1..N}$, i.e. find $\{V_i\}_{i=1}^r$ such that $\forall w \in W_{eq}, \exists c \in \mathbb{R}^r, w = \sum_i c_i V_i$ (and use $c$ as a parameter vector)


Let's do this for the cartpole

In [ ]:
def get_equivariant_basis(state_group: List[torch.TensorType], action_group: List[torch.TensorType], sample_size: int):
    """
    Get equivariant basis by finding the subspace of symmetrized samples of (non-equivariant weights)
    
    return 
    - a basis in the form of a tensor stacking R tensors of shape (2 x 4), with R the rank of the basis.
    - the rank of the basis (an integer)
    """
    raise NotImplementedError
    

state_group_reps = get_cartpole_state_group_representations()
action_group_reps = get_cartpole_action_group_representations()

basis, rank = get_equivariant_basis(state_group_reps, action_group_reps, sample_size=500)

print('basis:', basis)
print('rank:', rank)


**Questions**: 
- Have a look at the basis coefficients: what is the effect of the symmetrizer?
- was the rank of the basis expected?

Let's test this basis!
Build some equivariant weights from this basis and test the resulting weights are indeed equivariant

In [ ]:
# build some weights as a linear combination of the basis
weights = # your code here


# let's check if the associated network is indeed equivariant
z = torch.tensor(np.random.rand(4,1).astype(np.float32))
weights = torch.tensor(weights.astype(np.float32))

test_network_is_equivariant(weights, z, group) # test should pass!


## Going deep: Stacking equivariant layers

If we want to go deep, we can stack layers $f(x) = f_2(f_1(x))$.

How to do so to maintain the equivariance of the network?

You can use intermediate transformation $L_g$:


$
\begin{align} 
K_g [f (x)] &= K_g [f_2(f_1(x))]\\
&= f_2(P_g [f_1(x)]) \quad \text{($f_2$ equivariance constraint)}\\
&= f_2(f_1(L_g [x])) \quad \text{($f_1$ equivariance constraint)}\\
&= f (L_g [x]) 
\end{align}
$


here we can use $L_g=K_g$


In [ ]:
# Let's create a deep equivariant network f(x) = f_2(f_1(x))
# f_1(x) = sig(W_1 x + b_1)
# f_2(x) = sig(W_2 x + b_2)

# sample weights for layers 1 and 2
W_1 = torch.tensor(np.random.rand(2, 4).astype(np.float32))
W_2 = torch.tensor(np.random.rand(2, 2).astype(np.float32))
# sample random state
z = torch.tensor(np.random.rand(4,1).astype(np.float32))


# get equivariant weights using the symmetrizer
# we have already done the job for the first layer
group_1 = # your code
W_1_sym = # symmetrize weights W_1

# for the second layer, the input/output transformations are different
group_2 = # your code
W_2_sym = # symmetrize weights W_2




# now checking that the resulting deep network is equivariant
# I built f_2(f_1(x)) for you here.

b_1 = .01
b_2 = .01
sig = lambda x: torch.relu( x)

# compute f = f(x)
h1 = sig(torch.matmul(W_1_sym, z) + b_1)
f = sig(torch.matmul(W_2_sym, h1) + b_2)

# compute f_p = f(L_g(x))
L_g = group_1[0][1]
zp = torch.matmul(L_g, z)
h1p = sig(torch.matmul(W_1_sym, zp) + b_1) 
fp = sig(torch.matmul(W_2_sym, h1p) + b_2) 

# check that K_g f(L_g(x)) = f(x)
K_g = group_1[1][1]
all(torch.matmul(K_g, fp) == f) # test should pass!



## BONUS: Training an equivariant policy

We now have all the tools to build an equivariant policy!

If you have code in pytorch to run PG, you can plug the policy architecture you obtained!


Alternatively, you can expand the code base of Matteo with this particular policy class!



Finally, you can use the code that came with the paper! https://github.com/ElisevanderPol/mdp-homomorphic-networks

